# Introduction - Hyperparameters Search

In this notebook we demonstrate the use of **Word Embeddings (Word2Vec)** weighting technique into Information Retrieval to make trace link recovery between Test Cases and Bug Reports.

We model our study as follows:

* Each bug report title, summary and description compose a single query.
* We use each test case content as an entire document that must be returned to the query made

## Import Libraries

In [2]:
from mod_finder_util import mod_finder_util
mod_finder_util.add_modules_origin_search_path()

import pandas as pd
import numpy as np
import spacy

from tqdm import tqdm

from sklearn.externals.joblib import Parallel, delayed

from modules.utils import plots
from modules.utils import firefox_dataset_p2 as fd
from modules.utils import tokenizers as tok
from modules.utils import aux_functions
from modules.utils import model_evaluator as m_eval

from modules.models.wordvec import SimilarityMeasure
from modules.models.wordvec import WordVec_BasedModel
from modules.models.model_hyperps import WordVec_Model_Hyperp

import warnings; warnings.simplefilter('ignore')

## Load Dataset

In [3]:
test_cases_df = fd.Datasets.read_testcases_df()
bug_reports_df = fd.Datasets.read_selected_bug_reports_2_df()

corpus = test_cases_df.tc_desc
query = bug_reports_df.br_desc

test_cases_names = test_cases_df.tc_name
bug_reports_names = bug_reports_df.br_name

orc = fd.Tc_BR_Oracles.read_oracle_expert_volunteers_df()

TestCases.shape: (207, 12)
BugReports.shape: (93, 19)
Oracle.shape: (207, 93)


### Select Subset

In [5]:
bugreports_subset_df = bug_reports_df[(bug_reports_df.Version == '48 Branch') | (bug_reports_df.Version == '60 Branch')].sample(15, random_state=42)
testcases_subset_df = test_cases_df[(test_cases_df.TestDay.str.contains('20161014')) | (test_cases_df.TestDay.str.contains('20161028'))].sample(10, random_state=1000)

selected_testcases = ['TC_{}_TRG'.format(tc_num) for tc_num in [13, 14, 15, 16, 17, 18]]  # should link with 48 Branch
aux_tc = test_cases_df[test_cases_df.tc_name.isin(selected_testcases)]

tc_subset_df = testcases_subset_df.append(aux_tc)
tc_subset_df.drop_duplicates(inplace=True)

corpus_subset = tc_subset_df.tc_desc
query_subset = bugreports_subset_df.br_desc
testcases_names_subset = tc_subset_df.tc_name
bug_reports_names_subset = bugreports_subset_df.br_name
orc_subset_df = orc.loc[testcases_names_subset, bug_reports_names_subset]

print('TestCases Subset Shape: {}'.format(tc_subset_df.shape))
print('BugReports Subset Shape: {}'.format(bugreports_subset_df.shape))
print('Oracle Subset Shape: {}'.format(orc_subset_df.shape))

TestCases Subset Shape: (14, 10)
BugReports Subset Shape: (15, 12)
Oracle Subset Shape: (14, 15)


### Find The Best Model

In [11]:
all_hyperparams = {
    WordVec_Model_Hyperp.SIM_MEASURE_MIN_THRESHOLD.value : [('cosine' ,x)  for x in [.75,.85,.95]],
    WordVec_Model_Hyperp.TOP.value : [3,5],
    WordVec_Model_Hyperp.TOKENIZER.value : [tok.PorterStemmerBased_Tokenizer(), tok.LancasterStemmerBased_Tokenizer(), 
                                                   tok.WordNetBased_LemmaTokenizer(), tok.SnowballStemmerBased_Tokenizer()]
}

hyperparams = aux_functions.generate_params_comb_list(**all_hyperparams)          

print('Performing model hyperparameters search...')

results_df = pd.DataFrame(columns=['precision', 'recall', 'fscore', 'model_name', 'top_value', 'tokenizer', 'metric', 'metric_value', 'model_dump', 'evaluator_dump'])

#def run_model(idx, **hyperp):    
for idx,hp in tqdm(enumerate(hyperparams)):
    current_model = WordVec_BasedModel(**hp)
    current_model.set_name('WordVec_Based_Model_{}'.format(idx))
    current_model.recover_links(corpus_subset, query_subset, testcases_names_subset, bug_reports_names_subset)
    
    evaluator = m_eval.ModelEvaluator(orc_subset_df, current_model)
    evaluator.evaluate_model()
    evaluator.dump_model()
    evaluator.dump_evaluator()
    
    results_df = results_df.append(pd.DataFrame([[evaluator.get_mean_precision(), 
                    evaluator.get_mean_recall(),
                    evaluator.get_mean_fscore(), 
                    evaluator.get_model().get_name(),
                    evaluator.get_model().get_top_value(),
                    evaluator.get_model().get_tokenizer_type(),
                    evaluator.get_model().get_sim_measure_min_threshold()[0],
                    evaluator.get_model().get_sim_measure_min_threshold()[1],
                    evaluator.get_model().get_model_dump_path(),
                    evaluator.get_evaluator_dump_path()
           ]], columns=results_df.columns), ignore_index=True)

#tasks = [(idx,hp) for idx,hp in enumerate(hyperparams)]
#results = Parallel(n_jobs=-1, verbose=1)(delayed(run_model)(idx, **hp) for idx,hp in tasks)
results_df = results_df.astype(dtype={'model_dump' : str, 'evaluator_dump' : str, 'top_value': int})


0it [00:00, ?it/s]

Performing model hyperparameters search...



1it [00:10, 10.51s/it]
2it [00:21, 10.52s/it]
3it [00:32, 10.90s/it]
4it [00:43, 10.81s/it]
5it [00:54, 10.77s/it]
6it [01:04, 10.72s/it]
7it [01:15, 10.65s/it]
8it [01:25, 10.65s/it]
9it [01:36, 10.66s/it]
10it [01:47, 10.64s/it]
11it [01:58, 10.72s/it]
12it [02:08, 10.76s/it]
13it [02:19, 10.75s/it]
14it [02:31, 11.04s/it]
15it [02:42, 11.20s/it]
16it [02:54, 11.35s/it]
17it [03:06, 11.41s/it]
18it [03:17, 11.42s/it]
19it [03:28, 11.35s/it]
20it [03:40, 11.32s/it]
21it [03:50, 11.17s/it]
22it [04:02, 11.22s/it]
23it [04:13, 11.22s/it]
24it [04:24, 11.11s/it]


### Report

In [12]:
best_model = aux_functions.report_best_model(results_df)

------------ Report -------------------

Total of Analyzed Hyperparameters Combinations: 24

Best Model Hyperparameters Combination Found:

{'Measures': {'Mean FScore of WordVec_Based_Model_6': 0.16666666666666663,
              'Mean Precision of WordVec_Based_Model_6': 0.2,
              'Mean Recall of WordVec_Based_Model_6': 0.14285714285714285},
 'Setup': [{'Name': 'WordVec_Based_Model_6'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'Tokenizer': <utils.tokenizers.WordNetBased_LemmaTokenizer object at 0x7fb49aa33e80>}]}


### Save Similarity Matrix

In [13]:
best_model.save_sim_matrix()

#### Best Model for TOP 3 and 5 - Cosine

In [16]:
aux_functions.print_report_top_3_and_5_v3(results_df, metric=SimilarityMeasure.COSINE.value)

{'Measures': {'Mean FScore of WordVec_Based_Model_2': 0.14666666666666667,
              'Mean Precision of WordVec_Based_Model_2': 0.24444444444444444,
              'Mean Recall of WordVec_Based_Model_2': 0.10476190476190476},
 'Setup': [{'Name': 'WordVec_Based_Model_2'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 3},
           {'Tokenizer': <utils.tokenizers.WordNetBased_LemmaTokenizer object at 0x7fb405b6bf60>}]}
------------------------------------------------------------------
{'Measures': {'Mean FScore of WordVec_Based_Model_6': 0.16666666666666663,
              'Mean Precision of WordVec_Based_Model_6': 0.2,
              'Mean Recall of WordVec_Based_Model_6': 0.14285714285714285},
 'Setup': [{'Name': 'WordVec_Based_Model_6'},
           {'Similarity Measure and Minimum Threshold': ('cosine', 0.75)},
           {'Top Value': 5},
           {'Tokenizer': <utils.tokenizers.WordNetBased_LemmaTokenizer object at 0x7fb42e84